# Neural Network

In [125]:
import sklearn as sk
import numpy as np
import pandas as pd
import time
from sklearn.cross_validation import KFold
from sklearn import tree
from sklearn import ensemble
from sklearn import datasets
import theano.tensor as th
from scipy import misc
import copy
from numpy.random import uniform
from numpy.random import normal
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification

import matplotlib.pylab as plt
%matplotlib inline

In [156]:
class network:
    def __init__(self, layers_list, learning_rate, alpha, activation_functions, cost_func, mode, cou_iter):
        self.layers_count = len(layers_list)
        self.weight_list = []
        self.learning_rate = learning_rate
        self.layers_list = layers_list
        self.alpha = alpha
        self.cost_func = cost_func
        self.activation_functions = activation_functions
        self.mode = mode
        self.cou_iter = cou_iter
        for i in range(1, self.layers_count):
            m = np.asarray(normal(0, 0.15, (self.layers_list[i - 1] + 1, self.layers_list[i])))
            self.weight_list.append(m)
            
    def w_out(self):
        for i in range(len(self.weight_list)):
            print self.weight_list[i]
            print '\n'
    
    def sigmoid(self, x):
        one = np.ones(len(x))
        val = one / (one + np.exp(-x * self.alpha))
        for i in range(len(val)):
            if (val[i] == 0):
                val[i] = 0.00001
            elif (val[i] == 1):
                val[i] = 0.99999
        return one / (one + np.exp(-x * self.alpha))
    
    def der_sigmoid(self, x):
        one = np.ones(len(x))
        return (one - self.sigmoid(x)) * self.sigmoid(x) * self.alpha
    
    def x(self, x):
        return x
    
    def der_x(self, x):
        return np.ones(len(x))
    
    def hyp_tg(self, x):
        return (np.exp(self.alpha * x) - np.exp(-self.alpha * x)) / (np.exp(self.alpha * x) + np.exp(-self.alpha * x))
    
    def der_hyp_tg(self, x):
        return self.alpha * (1 - (self.hyp_tg(x)) ** 2)
    
    def logistic_cost(self, y_true, y_pred):
        val = 0
        if (self.mode == 'class'):
            z = np.zeros(len(y_pred))
            z[y_true] = 1
            one = np.ones(len(y_pred))
            y_true = z.copy()
            val = -np.sum(y_true * np.log(y_pred) + (one - y_true) * np.log(one - y_pred))
        elif (self.mode == 'reg'):
            val = -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return val
    
    def der_logistic_cost(self, y_true, y_pred):
        val = 0
        if (self.mode == 'class'):
            z = np.zeros(len(y_pred))
            z[y_true] = 1
            one = np.ones(len(y_pred))
            y_true = z
            val = ((one*1.0 - y_true*1.0) / (one - y_pred*1.0) - (y_true*1.0 / y_pred))
        elif (self.mode == 'reg'):
            val = ((1.0 - y_true) / (1.0 - y_pred) - y_true / y_pred)
        return val
    
    def square_cost(self, y_true, y_pred):
        return  0.5*((y_true - y_pred) ** 2.0)
    
    def der_square_cost(self, y_true, y_pred):
        return (y_pred - y_true)
    
    def add_first_layer(self, x_shape):
        self.weight_list.insert(0, np.asarray(normal(0, 0.15, (x_shape, self.layers_list[0]))))
    
    def predict(self, x):
        x = np.asarray(x)
        x = np.insert(x, 0, [-1], axis = 1)
        ans = []
        for s_num, sample in enumerate(x):
            v = sample
            for num, layer in enumerate(self.weight_list):
                v = np.dot(layer.T, v)
                if (self.activation_functions[num] == 'sigmoid'):
                    v = self.sigmoid(v)
                elif (self.activation_functions[num] == 'hyp_tg'):
                    v = self.hyp_tg(v)
                elif (self.activation_functions[num] == 'x'):
                    v = self.x(v)
                    
                if (num != self.layers_count - 1):
                    v = np.insert(v, 0, [-1])
                    
            if (self.mode == 'class'):
                ans.append(np.argmax(v))
            elif (self.mode == 'reg'):
                ans.append(v)
        return ans
    
    
    def fit(self, x, y):
        y = np.asarray(y)
        x = np.insert(np.asarray(x), 0, -1, axis = 1)
        self.add_first_layer(x.shape[1])
        k = 0
        total_err = 0.0
        prev_err = 0.0
        while (k < self.cou_iter):
            print k, total_err / len(x)
            total_err = 0.0
            x, y = shuffle(x, y)
            for s_num, sample in enumerate(x):
                neuron_sum = []
                neuron_out = []
                neuron_out.append(np.asarray(sample, dtype = np.float))
                v = sample
                for num, layer in enumerate(self.weight_list):
                    v = np.dot(layer.T, v)
                    neuron_sum.append(v)
                    
                    if (self.activation_functions[num] == 'sigmoid'):
                        v = self.sigmoid(v)
                    elif (self.activation_functions[num] == 'hyp_tg'):
                        v = self.hyp_tg(v)
                    elif(self.activation_functions[num] == 'x'):
                        v = self.x(v)
                        
                    if (num != self.layers_count - 1):
                        v = np.insert(v, 0, [-1])
                    neuron_out.append(v)

                if (self.cost_func == "logistic"):
                    total_err += self.logistic_cost(y[s_num], v)
                elif (self.cost_func == "square"):
                    total_err += self.square_cost(y[s_num], v)
                    
                if (neuron_out[-1].shape[0] == 1):
                    neuron_out[-1] = neuron_out[-1][0]

                if (self.cost_func == "logistic" and self.activation_functions[-1] == "sigmoid"):
                    der_neuron = [(self.der_logistic_cost(y[s_num], neuron_out[-1]) * self.der_sigmoid(neuron_sum[-1]))]
                elif (self.cost_func == "logistic" and self.activation_functions[-1] == "hyp_tg"):
                    der_neuron = [(self.der_logistic_cost(y[s_num], neuron_out[-1]) * self.der_hyp_tg(neuron_sum[-1]))]
                elif (self.cost_func == 'logistic' and self.activation_functions[-1] == "x"):
                    der_neuron = [(self.der_logistic_cost(y[s_num], neuron_out[-1]) * self.der_x(neuron_sum[-1]))]
                elif(self.cost_func == "square" and self.activation_functions[-1] == "sigmoid"):
                    der_neuron = [(self.der_square_cost(y[s_num], neuron_out[-1]) * self.der_sigmoid(neuron_sum[-1]))]
                elif (self.cost_func == "square" and self.activation_functions[-1] == "hyp_tg"):
                    der_neuron = [(self.der_square_cost(y[s_num], neuron_out[-1]) * self.der_hyp_tg(neuron_sum[-1]))]
                elif (self.cost_func == 'square' and self.activation_functions[-1] == "x"):
                    der_neuron = [(self.der_square_cost(y[s_num], neuron_out[-1]) * self.der_x(neuron_sum[-1]))]
                
                num = 1
                for layer in reversed(self.weight_list[1:]):
                    if (self.activation_functions[self.layers_count - num - 1] == 'sigmoid'):
                        der_neuron.insert(0, np.dot(layer[1:], der_neuron[0]) * self.der_sigmoid(neuron_sum[len(neuron_sum) - num - 1]))
                    elif (self.activation_functions[self.layers_count - num - 1] == 'hyp_tg'):
                        der_neuron.insert(0, np.dot(layer[1:], der_neuron[0]) * self.der_hyp_tg(neuron_sum[len(neuron_sum) - num - 1]))
                    elif (self.activation_functions[self.layers_count - num - 1] == 'x'):
                        der_neuron.insert(0, np.dot(layer[1:], der_neuron[0]) * self.der_x(neuron_sum[len(neuron_sum) - num - 1]))
                    num += 1

                for layer_num in range(self.layers_count):
                    self.weight_list[self.layers_count - layer_num - 1] -= self.learning_rate * np.dot(neuron_out[len(neuron_out) - layer_num - 2].T.reshape(-1, 1), der_neuron[len(der_neuron) - layer_num - 1].reshape(1, -1))
                
            k += 1

            
            
                    

In [164]:


iris = load_iris()
x = iris.data
y = iris.target

x, y = shuffle(x, y)

xtest = x[100:]
ytest = y[100:]
x = x[:100]
y = y[:100]

nt = network([4, 3], 0.05, 1.0, ['sigmoid', 'sigmoid'], "logistic", 'class', 80)
nt.fit(x, y)
ypred = nt.predict(xtest)
print ypred, ytest
print accuracy_score(ytest, ypred)
#print mean_squared_error(ytest, ypred)
#print accuracy_score(ytest, ypred)


0 0.0
1 1.93438055439
2 1.81044757852
3 1.60767345038
4 1.38555067452
5 1.23462052716
6 1.14259943018
7 1.09013892983
8 1.05306442421
9 1.02750599857
10 0.996383613804
11 0.979256104786
12 0.959838219328
13 0.939818147334
14 0.91284612921
15 0.895815631895
16 0.854812038148
17 0.838361427404
18 0.814124543936
19 0.757688125006
20 0.77223496318
21 0.745474615183
22 0.700497526541
23 0.692683154375
24 0.592509308765
25 0.579377305683
26 0.576736478158
27 0.626732300947
28 0.519751873895
29 0.49969633287
30 0.528983677422
31 0.535450747741
32 0.481876631239
33 0.442270674282
34 0.440007931893
35 0.424049132939
36 0.415895396352
37 0.314096107293
38 0.307307920029
39 0.386612201286
40 0.347399008929
41 0.404270492514
42 0.359582051843
43 0.407808775477
44 0.434412160524
45 0.414995999966
46 0.351172728787
47 0.347806204455
48 0.338574281265
49 0.387744309816
50 0.356228032838
51 0.357586355038
52 0.323116572734
53 0.341708384959
54 0.266586947673
55 0.284293175515
56 0.258111116752
57 0.27

In [182]:
x, y = make_classification(n_samples=500, n_features=20, n_informative=10, n_classes=4)
xtest = x[450:]
ytest = y[450:]
x = x[:450]
y = y[:450]
nt = network([35, 4], 0.05, 1.0, ['sigmoid', 'sigmoid'], "logistic", 'class', 50)
nt.fit(x, y)

ypred = nt.predict(xtest)
print accuracy_score(ypred, ytest)


0 0.0
1 2.01112921213
2 1.74522136345
3 1.70031456916
4 1.59102015973
5 1.51342838849
6 1.42066742006
7 1.36592808488
8 1.2823206414
9 1.24043631222
10 1.15316337836
11 1.09237304983
12 1.01304506898
13 0.957162961383
14 0.899558727482
15 0.830984624595
16 0.77615343063
17 0.717810607361
18 0.690549854836
19 0.650896442631
20 0.603710602736
21 0.58276547746
22 0.538242210883
23 0.508650668871
24 0.473376103828
25 0.454573290395
26 0.42378583151
27 0.397535532885
28 0.387302423728
29 0.354256667785
30 0.336588558836
31 0.319891997422
32 0.301829644837
33 0.279941123152
34 0.286660323065
35 0.271133785968
36 0.250289929865
37 0.236539738248
38 0.219923991336
39 0.206209307513
40 0.196341985298
41 0.186685355619
42 0.175228983116
43 0.171413494913
44 0.163566426328
45 0.165666704412
46 0.153239299292
47 0.145164197968
48 0.134903553682
49 0.135234904528
0.66


In [183]:
im = misc.imread('data/big_alphabet_29x29/mutant-0-0-0.bmp', flatten='grey')
alphabet_size = 26
im_size = 29

In [184]:
x = []
y = []
x_test = []
for letter in range(alphabet_size):
    for i in range(8):
        path = "data/big_alphabet_29x29/mutant-" + str(letter) + "-" + str(i) + "-0.bmp"
        im = misc.imread(path, flatten='grey')
        x.append(im.reshape(im_size * im_size))
        x[-1] /= 255
        y.append(letter)
for letter in range(alphabet_size):
    path = "data/big_alphabet_29x29/class-" + str(letter) + ".bmp"
    im = misc.imread(path, flatten='grey')
    x_test.append(im.reshape(im_size * im_size))
    x_test[-1] /= 255
ytest = np.arange(26)

In [185]:
 x = np.asarray(x)
print x
print x.shape
print len(x_test)
print y

[[ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 ..., 
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]]
(208, 841)
26
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25]


In [195]:
#x, y = shuffle(x, y)
nt = network([43, 26], 0.01, 1.0, ['sigmoid', 'sigmoid'], "logistic", 'class', 30)
nt.fit(x, y)

ypred = nt.predict(x_test)
print ypred
print accuracy_score(ytest, ypred)

0 0.0
1 5.30018119262
2 4.29254079772
3 4.25095791426
4 4.17075803876
5 4.08203850047
6 3.97057511452
7 3.86294852432
8 3.76102912292
9 3.62805555467
10 3.48462689116
11 3.36155418134
12 3.23186730256
13 3.09992682007
14 2.98663219579
15 2.85914997825
16 2.75768086141
17 2.63622027272
18 2.53050710694
19 2.43160023088
20 2.32970837989
21 2.22744003054
22 2.12950408124
23 2.04564722684
24 1.96377709265
25 1.8740856477
26 1.80379911061
27 1.72848318035
28 1.6625024167
29 1.59428309599
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
1.0
